In [22]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Activation,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.models import load_model
import cv2
from PIL import Image
import numpy as np

In [2]:
batch_size= 32
classes = 6
rows,columns=48,48

train_dir = 'fer2013\\train' 
validation_dir = 'fer2013\\validation'

train_gen = ImageDataGenerator(rescale=1./255,
                              rotation_range=45,
                              height_shift_range=0.3,
                              width_shift_range=0.3,
                              shear_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')

validation_gen = ImageDataGenerator(rescale=1./255)

training_data_generator = train_gen.flow_from_directory(train_dir,
                                                       batch_size=batch_size,
                                                       shuffle=True,
                                                       color_mode='grayscale',
                                                       class_mode='categorical',
                                                       target_size=(rows,columns))

valiadtion_data_generator = validation_gen.flow_from_directory(validation_dir,
                                                       batch_size=batch_size,
                                                       shuffle=False,
                                                       color_mode='grayscale',
                                                       class_mode='categorical',
                                                       target_size=(rows,columns))




Found 28273 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.


## Creating Our Model

In [3]:
model = Sequential()

# First Block
model.add(Conv2D(64,(3,3),activation='elu',input_shape=(rows,columns,1),kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='elu',input_shape=(rows,columns,1),kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Second Block
model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Third Block
model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Fourth Block
model.add(Conv2D(512,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Fifth Block
model.add(Flatten())
model.add(Dense(256,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Sixth Block
model.add(Dense(128,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Seventh Block
model.add(Dense(64,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Eighth Block
model.add(Dense(classes,activation='softmax',kernel_initializer='he_normal'))

print(model.summary())

W1114 19:11:53.931808  9912 deprecation_wrapper.py:119] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)      

In [53]:
checkpoint = ModelCheckpoint('model\\6_class_emotion_detector_V1.h5',
                             save_best_only=True,
                             mode='min',
                             monitor='val_loss',
                             verbose=1)

earlystopping = EarlyStopping(patience=3,
                             verbose=1,
                             min_delta=0,
                             monitor='val_loss',
                             restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              verbose=1,
                              min_delta=0.0001,
                              patience=3)

callbacks = [checkpoint,earlystopping,reduce_lr]

model.compile(metrics=['accuracy'],
             optimizer=RMSprop(lr=0.0001),
             loss='categorical_crossentropy')

train_samples = 28273
validation_samples = 3534
batch_size = 32
epochs=30

# Training Our Model

In [6]:
history = model.fit_generator(training_data_generator,
                              epochs=epochs,
                              steps_per_epoch=train_samples//batch_size,
                              validation_data=valiadtion_data_generator,
                              validation_steps=validation_samples//batch_size,
                              callbacks=callbacks)

Epoch 1/30
883/883 [==============================] - 1354s 2s/step - loss: 2.2646 - accuracy: 0.1920 - val_loss: 1.9375 - val_accuracy: 0.2565

Epoch 00001: val_loss did not improve from 1.88453
Epoch 2/30
883/883 [==============================] - 1356s 2s/step - loss: 2.0995 - accuracy: 0.1982 - val_loss: 1.9575 - val_accuracy: 0.2601

Epoch 00002: val_loss did not improve from 1.88453
Epoch 3/30
883/883 [==============================] - 1314s 1s/step - loss: 1.9995 - accuracy: 0.2014 - val_loss: 1.7097 - val_accuracy: 0.2587

Epoch 00003: val_loss improved from 1.88453 to 1.70970, saving model to model\6_class_emotion_detector_V1.h5
Epoch 4/30
883/883 [==============================] - 1362s 2s/step - loss: 1.9202 - accuracy: 0.2097 - val_loss: 1.5180 - val_accuracy: 0.2670

Epoch 00004: val_loss improved from 1.70970 to 1.51798, saving model to model\6_class_emotion_detector_V1.h5
Epoch 5/30
883/883 [==============================] - 1383s 2s/step - loss: 1.8634 - accuracy: 0.216

In [52]:
model = load_model('model\\6_class_emotion_detector_V1.h5')


# Live Predictor

In [76]:
emotions = {0:'Angry',1:'Fear',2:'Happy',3:'Neutral',4:'Sad',5:'Surprise'}

cap = cv2.VideoCapture(0)

classifier = cv2.CascadeClassifier('Haarcascades\\haarcascade_frontalface_default.xml')

def detect_face(frame):
    faces=classifier.detectMultiScale(frame,1.3,4)
    if faces==():
        return frame
    for x,y,w,h in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(172,42,251),2)
        face = frame[y:y+h,x:x+w]
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        face = cv2.resize(face,(48,48))
        face = face.reshape(1,48,48,1)
        print (emotions[np.argmax(model.predict(face))])
        cv2.putText(frame,text=emotions[np.argmax(model.predict(face))],
                    org=(x,y-15),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(106,40,243),thickness=2)
    return frame

while 1:
    ret,frame= cap.read()
    if ret==True:
        cv2.imshow('emotion_detector',detect_face(frame))
        if cv2.waitKey(1)==27:
            break
cap.release()
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy


# Predicting from Image

In [92]:
image_path = "D:\\Pictures\\New folder\\pics\\IMG_20151229_151631.jpg"
img = cv2.imread(image_path)
# just change these ratios if the resultant image is too small or not picking up the faces
# img = cv2.resize(img,None,fx=0.5,fy=0.5)  
cv2.imshow('emotions_from_image ',detect_face(img))
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


Neutral
Happy
Happy
Neutral
Happy
Happy
Neutral
Happy
